In [1]:
import numpy as np
import tncontract as tn
import qutip as qt
import time
from itertools import product
from functools import reduce

from scipy.stats import cumfreq
from copy import deepcopy
from IPython.display import clear_output

from src.simulation_utils import *
from src.plotting_utils import *

In [17]:
n_sites = 6
X = qt.sigmax()
I = qt.qeye(2)
XX = qt.tensor(X, X)
H = sum([local_operator(i, XX, n_sites) for i in range(n_sites-1)])
zero = qt.operator_to_vector(qt.ket2dm(reduce(qt.tensor, [qt.basis(2,0)] * n_sites)))
# one = reduce(qt.tensor, [qt.basis(2,1)] * n_sites)

# Ising-like background noise
background = 1e-4
crosstalk_ham = sum([background * np.random.randn() * local_operator(i, XX, n_sites) for i in range(n_sites-1)])

# some bigger systematic nonlocal couplings
systematic = 1e-2
crosstalk_ham += systematic * np.random.randn() * reduce(qt.tensor, [I,X,I,I,X,I])#,I])
crosstalk_ham += systematic * np.random.randn() * reduce(qt.tensor, [I,I,X,I,X,I])#,I])
crosstalk_ham += systematic * np.random.randn() * reduce(qt.tensor, [X,I,I,I,I,X])#,X])

crosstalk = qt.to_super(qt.propagator(crosstalk_ham, 3*np.pi/2))

In [18]:
crosstalk

Quantum object: dims = [[[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]], [[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]]], shape = (4096, 4096), type = super, isherm = False
Qobj data =
[[ 9.95470155e-01+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j ...  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  1.44586662e-20+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j  9.95470155e-01+0.00000000e+00j
   5.74068318e-13-3.75226029e-04j ... -1.57016242e-21+3.15619968e-17j
   1.44586662e-20+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 0.00000000e+00+0.00000000e+00j  5.74068318e-13-3.75226029e-04j
   9.95470155e-01+0.00000000e+00j ...  1.44586662e-20+0.00000000e+00j
  -1.57016242e-21+3.15619968e-17j  0.00000000e+00+0.00000000e+00j]
 ...
 [ 0.00000000e+00+0.00000000e+00j -1.57016242e-21+3.15619968e-17j
   1.44586662e-20+0.00000000e+00j ...  9.95470155e-01+0.00000000e+00j
   5.74068318e-13-3.75226029e-04j  0.00000000e+00+0.00000000e+00j]
 [ 0.000000

In [19]:
dep = (.99) * np.eye(4 ** n_sites, 4 ** n_sites) 
dep[0,0] = 1
dep_super = ptm_to_super(dep)
depolarising = qt.Qobj(dep_super, dims=crosstalk.dims, type=super)

In [20]:
depolarising

Quantum object: dims = [[[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]], [[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]]], shape = (4096, 4096), type = super, isherm = True, superrep = None
Qobj data =
[[9.9015625e-01 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 1.5625000e-04]
 [0.0000000e+00 9.9000000e-01 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 9.9000000e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 9.9000000e-01
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  9.9000000e-01 0.0000000e+00]
 [1.5625000e-04 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 9.9015625e-01]]

In [21]:
specs = []
depths = []
for j in range(cct_depth):
    one_qubit_layer = random_single_qubit_gate_layer(n_sites)
    two_qubit_layer = random_two_qubit_gate_ladder(n_sites)[np.mod(j, 2)]

    psi = tn.onedim.contract_mps_mpo(psi, one_qubit_layer)
    psi.left_canonise(chi=chi)

    psi = tn.onedim.contract_mps_mpo(psi, two_qubit_layer)
    psi.left_canonise(chi=chi)

    if np.mod(2*j, slice_separation) == 0:
        depths.append(j)
        phi = deepcopy(psi)
        phi.left_canonise(normalise=True)

        phi_vec = tn.onedim.contract_virtual_indices(phi)
        phi_vec.fuse_indices('physout','physout')

        prob_vec = abs(phi_vec.data.reshape(-1, 1)) ** 2

        specs.append(prob_vec)
#         clear_output(wait=True)
#         speckle_pattern(prob_vec)
#         plt.show()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


KeyboardInterrupt: 

In [ ]:
slice_separation = 200